In [1]:
from IPython.display import display, clear_output
#import IPython.display as ipyds
import ipyvuetify as v
import pandas as pd
#import numpy as np
import ipywidgets as widgets
import time as t
#import matplotlib.pylab as plt
from run.hydration import bouges_composition
from run.hydration import parrot_killoh
from run.hydration import run_hydration
from run.hydration import plot_bars
from run.hydration import phase_plot
from run.hydration import to_phase_first_dict
#import plotly.express as px
import plotly.graph_objects as go
#from ipysheet import from_dataframe

In [2]:
#import matplotlib as mpl
#mpl.rcParams['figure.figsize']=[15,15]

In [3]:
def make_dropdown(label_, dic_):
    return v.Select(label=label_, v_model = "",
    items=dic_[label_], style_='width: 250px; opacity: 1')

#parameters for parrot and killoh model source: lothenbach et al. 2008
K1 = {"C3S":1.5,"C2S":0.5,"C3A":1.,"C4AF":0.37}
N1 = {"C3S":0.7,"C2S":1.0,"C3A":0.85,"C4AF":0.7}
K2 = {"C3S":0.05,"C2S":0.02,"C3A":0.04,"C4AF":0.015}
K3 = {"C3S":1.1,"C2S":0.7,"C3A":1.0,"C4AF":0.4}
N3 = {"C3S":3.3,"C2S":5.0,"C3A":3.2,"C4AF":3.7}
H  = {"C3S":1.8,"C2S":1.35,"C3A":1.6,"C4AF":1.45}
Ea = {"C3S":41570,"C2S":20785,"C3A":54040,"C4AF":34087} #from barbara's excel sheets in paper only 2 leading digits are given
T0 = 20#c
ref_fineness = 385

def parrot_killoh():
    DoH = {}
    for phase in ["C3S","C2S","C3A","C4AF"]:
        ivp_out = solve_ivp(overall_rate, [0,np.max(output_times)], [1e-15],t_eval=output_times
                           ,args=(K1[phase],N1[phase],K2[phase],K3[phase],N3[phase],
                           H[phase],wc,RH,fineness,ref_fineness,T0,Ea[phase]))
        DoH[phase] = np.squeeze(ivp_out.y[0])
    return DoH

In [4]:
# models selection
GeoChemProcesses = {'Geochemical Processes':  ['Cement Hydration', 'Cement Carbonation', 'Metal Corrosion', 'Organics Degradation']}
dropdownGeoChemProcesses = make_dropdown('Geochemical Processes', GeoChemProcesses)
MechProcesses = {'Mechanical Processes': ['Porosity/Permeability Evolution', 'Strain Field Evolution', 'Other...']}
dropdownMechProcesses = make_dropdown('Mechanical Processes', MechProcesses)
SurogateProcesses = {'Surogate modeling': ['Train Surogate Models', 'Test Surogate Models']}
dropdownSurogateProcesses = make_dropdown('Surogate modeling', SurogateProcesses)
#dropdownProcesses

In [5]:
container_dropdown = v.Container(_metadata={'mount_id': 'content-nav'}, class_='px-1 mx-0', 
                                 children=[dropdownGeoChemProcesses, dropdownMechProcesses, dropdownSurogateProcesses])

In [6]:
#input list
'''
CaO=64.18
SiO2=21.01
Al2O3=4.63
Fe2O3=2.6
SO3=2.78
MgO=1.82
CSH2=2.2 #gypsum 
'''
form_input_cement = v.Form(label='input',  class_='px-0 mx-5', children=[v.Container(children=[
    v.Row(children=[
        v.TextField(v_model='oxide composition in weight %', filled = True, disabled=True,
                    label="Cement composition", value = 'info')
    ]),
    v.Row(children=[
        v.TextField(label="CaO", value = '64.18', v_model = '64.18')
    ]),
    v.Row(children=[
        v.TextField(label="SiO2", value = '21.01', v_model = '21.01')
    ]),
    v.Row(children=[
        v.TextField(label="Al2O3", value = '4.63', v_model = '4.63')
    ]),
    v.Row(children=[
        v.TextField(label="Fe2O3", value = '2.6', v_model = '2.6')
    ]),
    v.Row(children=[
        v.TextField(label="SO3", value = '2.78', v_model = '2.78')
    ]),
    v.Row(children=[
        v.TextField(label="MgO", value = '1.82', v_model = '1.82')
    ]),
    v.Row(children=[
        v.TextField(label="Gypsum", value = '2.2', v_model = '2.2')
    ])
])])

# input other
form_input_other = v.Form(label='input-other', class_='px-0 mx-10', children=[v.Container(children=[
    v.Row(children=[
        v.TextField(v_model='input for the model mix composition', filled = True, disabled=True,
                    label="Other input", value = 'info')
    ]),
    v.Row(children=[
        v.TextField(label="water/cement", value = '0.5', v_model = '0.5')
    ]),
    v.Row(children=[
        v.TextField(label="Relative-Humidity", value = '1', v_model = '1')
    ]),
        v.Row(children=[
        v.TextField(label="Temperature C", value = '45', v_model = '45', disabled=True)
    ]),
        v.Row(children=[
        v.TextField(label="Fineness", value = '385', v_model = '385', disabled=True)
    ])
])])

In [7]:
output_plot1 = widgets.Output(layout = { 'min_width': '800px', 'height': '100%'})
#output_plot2 = widgets.Output(layout = { 'min_width': '800px', 'height': '100%'})

layout = {
            'width': '100%',
            'height': '500px',
            'border': '1px solid black'
        }

In [8]:
#plot_container = v.Container(children=[output_plot1 ])

In [9]:
def make_tabs(inputs, outputs):
    return v.Tabs( class_='mx-4', children=[
           v.Tab(children=['Process Input']),
           v.Tab(children=['Process Results']),
           v.TabItem(children=[v.Layout(row =True, wrap=True, align_top=True, 
                        children=[v.Flex(xs12=True, lg6=True, xl4=False, children=[input_]) for input_ in inputs ]) ]),
           v.TabItem(children=[v.Layout(row=True, wrap=True, align_center=True, 
                        children=[v.Flex(xs12=True, lg6=True, xl4=True, class_='px-2 mx-0', children=[output_]) for output_ in outputs ]) ])
           ])

In [10]:
tabs_hydration = make_tabs([form_input_cement, form_input_other], [output_plot1])
tabs_carbonation = make_tabs([], [])
#tabs

In [11]:
def make_card_process(title_, subtitle_, tabs_):
    return v.Card( children = [v.CardTitle(children=title_),
                               v.CardSubtitle(children=subtitle_), 
                               v.Layout( row=True, wrap=True, align_center=True, children=[
                                        v.Flex(xs12=True, lg6=True, xl4=True, children=[
                                            tabs_]) ]),
                               v.CardActions(class_='pl-4 ml-4', children=[ 
                                        v.Btn(color='primary', children=['Run Process']) ]) ], 
                    elevation="2", outlined=True, loading=False)

In [12]:
# Other process card
card_process_trainNN = v.Card(children = [
    v.CardTitle(children="Train Cement Hydration surogate model"),
    v.CardSubtitle(children="here the user could setup and train a cement hydration surogate model")
], elevation="2", outlined=True)

card_process_testNN = v.Card(children = [
    v.CardTitle(children="Test trained surogate models"),
    v.CardSubtitle(children="Here the user could test/run surogate models for different eveolution processes")
], elevation="2", outlined=True)

card_process_mechanics = v.Card(children = [
    v.CardTitle(children="Mechanical properties"),
    v.CardSubtitle(children="here the user could run a process on the mechanical properties of the waste pacakge that uses some output that from previous (chemical) processes")
], elevation="2", outlined=True)

# Cement hydration process card
card_process_cem_hydration = make_card_process('Cement hydration', 
                                           'OPC cement paste hydration process using Parrot and Killoh hydration model, GEMS and CEMDATA18',
                                           tabs_hydration)
# carbonation
card_process_cem_crabonation = make_card_process('Cement Carbonation', 
                                           'Model to run cement carbonation ...',
                                           tabs_carbonation)
card_process_met_corrosion = make_card_process('Metal Corrosion', 
                                           'Model to run Metal Corrosion ...',
                                           tabs_carbonation)
card_process_org_degradation = make_card_process('Organics Degradation', 
                                           'Model to run Organics Degradation ...',
                                           tabs_carbonation)

# main card added to the web-widget
#card_main = v.Card(_metadata={'mount_id': 'content-main'}, children = [])
container_main = v.Container(_metadata={'mount_id': 'content-main'}, class_='pl-4 ml-4', children=[])

In [13]:
def check(list1, val):
    for e in list1:
        if e > val:
            return True
    return False

In [14]:
# Run simulation click
count = 0

def on_click(widget, event, data):
    global count
    #amount = form_input_cement.children[0].children[count].children[0].v_model
    #card_process_hydration.children[3].children[0].children=[f'Click me {amount}']
    card_process_cem_hydration.loading = True
    #t.sleep(1)
    #count += 1
    
    CaO=float(form_input_cement.children[0].children[1].children[0].v_model)
    SiO2=float(form_input_cement.children[0].children[2].children[0].v_model)
    Al2O3=float(form_input_cement.children[0].children[3].children[0].v_model)
    Fe2O3=float(form_input_cement.children[0].children[4].children[0].v_model)
    SO3=float(form_input_cement.children[0].children[5].children[0].v_model)
    MgO=float(form_input_cement.children[0].children[6].children[0].v_model)
    CSH2=float(form_input_cement.children[0].children[7].children[0].v_model)
    
    wc = float(form_input_other.children[0].children[1].children[0].v_model)
    RH = float(form_input_other.children[0].children[2].children[0].v_model)
    T = float(form_input_other.children[0].children[3].children[0].v_model)
    fineness = float(form_input_other.children[0].children[4].children[0].v_model)
    
    clinker_phases = bouges_composition(CaO,SiO2,Al2O3,Fe2O3,SO3)
    
    pk = parrot_killoh(wc, RH, T, fineness)
    
    try:
        vol_frac, mass_frac, density = run_hydration(clinker_phases, wc, CSH2, T, pk)
    except Exception as e:
        print(e)
    
    with output_plot1:         
        clear_output(wait=True)
        gems_vol_frac_phase_first = to_phase_first_dict(vol_frac)
        gems_masses_phase_first = to_phase_first_dict(mass_frac)
        #p2 = phase_plot(gems_vol_frac_phase_first)
        #p2.show()
        
        x= [0.,0.1,1,3,7,28,30,60,90,365, 730,1000,2000]
        #['aq_gen', 'C3(AF)S0_84H', 'CSHQ', 'Belite']
        
        size = 0

        fig = go.Figure()
        for g in gems_vol_frac_phase_first:
            if  check(gems_vol_frac_phase_first[g], 1e-3):#np.count_nonzero(gems_vol_frac_phase_first[g]) > 0:
                size=size+1
                fig.add_trace(go.Scatter(
                    x=x, y=gems_vol_frac_phase_first[g],
                    hoverinfo='x+y',
                    name=g,
                    mode='lines',
                    line=dict(width=0.5), #color='rgb(131, 90, 241)'),
                    stackgroup='one', # define stack group
                    hovertemplate='%{y}'
                ))
                
        for g in gems_masses_phase_first:
            if  check(gems_masses_phase_first[g], 1e-3):#np.count_nonzero(gems_vol_frac_phase_first[g]) > 0:
                size=size+1
                fig.add_trace(go.Scatter(
                    x=x, y=gems_masses_phase_first[g],
                    hoverinfo='x+y',
                    name=g,
                    mode='lines',
                    line=dict(width=0.5), #color='rgb(131, 90, 241)'),
                    stackgroup='two', # define stack group
                    hovertemplate='%{y}',
                    visible=False
                ))  


        fig.update_xaxes(type="log")
        fig.update_layout(  yaxis_range=(0, 1), 
                            title = "volume fraction",
                            xaxis_title="Time [days]",
                            yaxis_title="Volume fraction [m^3/m^3 of initial volume]",
                            legend_title="Phases",
                            hovermode='x',     
                              updatemenus=[
                dict(
                buttons=list([
                    dict(
                        #args=["type", "volfrac"],q.index(v) if v in q else 99999 for v in vm
                        args = [{"visible": [ True if x<size/2 else False for x in range(size) ]}, 
                                {"xaxis.type": "log", "yaxis.range" : [0, 1], 
                                 "yaxis.title": "Volume fraction [m^3/m^3 of initial volume]", 
                                 "xaxis.title": "Time [days]", "title": "volume fraction"}],
                        label="Volume Fraction",
                        method="update"
                        ),
                    dict(
                        #args=["type", "masfrac"],
                        args = [{"visible": [ False if x<size/2 else True for x in range(size) ]},
                                {"xaxis.type": "log", "yaxis.range" : [0, 0.1435], 
                                 "yaxis.title": "mass [g/100 g of cement]", 
                                 "xaxis.title": "Time [days]", "title": "mass fraction"}],
                        label="Mass Fraction",
                        method="update"
                    )
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=1.0,
                xanchor="left",
                y=1.15,
                yanchor="top"
                )
        ])
        
        #pd_vol_frac = pd.DataFrame.from_dict(vol_frac)
        
        fig.show()
        
        #ipyds.display(from_dataframe(pd_vol_frac))
        
        #tabs.v_model='tab1'
        
    t.sleep(1)
    
    card_process_cem_hydration.loading = False
    
    #fig1.layout.width = 'auto'
    #fig1.layout.height = 'auto'
    #fig1.layout.min_height = '300px' # so it shows nicely in the notebook
    
card_process_cem_hydration.children[3].children[0].on_event('click', on_click)

In [15]:
#vol_frac.head() 'Cement Hydration', 'Mechanical evolution', 'Train Surogate Models', 'Test Surogate Models']

In [16]:
card_process_cem_hydration

Card(children=[CardTitle(children=['Cement hydration']), CardSubtitle(children=['OPC cement paste hydration pr…

In [17]:
def dropdownGeoChemProcesses_eventhandler(change):
    if (change.new == "Cement Hydration"):
        container_main.children = [card_process_cem_hydration]
    elif (change.new == "Cement Carbonation"):
        container_main.children = [card_process_cem_crabonation]
    elif (change.new == "Metal Corrosion"):
        container_main.children = [card_process_met_corrosion]
    elif (change.new == "Organics Degradation"):
        container_main.children = [card_process_org_degradation]
    #with output:
        #clear_output(wait=True)
        #display(c)
#output = widgets.Output(_metadata={'mount_id': 'content-main'})

In [18]:
dropdownGeoChemProcesses.observe(dropdownGeoChemProcesses_eventhandler, names='v_model')

In [19]:
display(dropdownGeoChemProcesses)
#card_main.loading=True
container_main

Select(items=['Cement Hydration', 'Cement Carbonation', 'Metal Corrosion', 'Organics Degradation'], label='Geo…

Container(class_='pl-4 ml-4')